In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [3]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("stories.pdf")
pdf_text =loader.load()
print(pdf_text)

[Document(metadata={'source': 'stories.pdf', 'page': 0}, page_content='THE WIND AND THE SUN  \nOnce the Wind and the Sun had an argument. “I am stronger than you,” said the Wind. “No, you are \nnot,” said the Sun. Just at that moment they saw a traveler walking across the road. He was wrapped \nin a shawl. The Sun and the Wind agreed that whoever could separate the traveller from his shawl \nwas stronger. The Wind took the first turn. He blew with all his might to tear the traveller’s shawl \nfrom his shoulders. But the harder he blew, the tighter the traveller gripped the shawl to his body. \nThe struggle wen t on till the Wind’s turn was over. Now it was the Sun’s turn. The Sun smiled warmly. \nThe traveller felt the warmth of the smiling Sun. Soon he let the shawl fall open. The Sun’s smile grew \nwarmer and warmer... hotter and hotter. Now the traveller no longe r needed his shawl. He took it off \nand dropped it on the ground. The Sun was declared stronger than the Wind.  \nMoral:

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap =200)
pdf_chunk = text_splitter.split_documents(pdf_text)
pdf_chunk

[Document(metadata={'source': 'stories.pdf', 'page': 0}, page_content='THE WIND AND THE SUN  \nOnce the Wind and the Sun had an argument. “I am stronger than you,” said the Wind. “No, you are \nnot,” said the Sun. Just at that moment they saw a traveler walking across the road. He was wrapped \nin a shawl. The Sun and the Wind agreed that whoever could separate the traveller from his shawl \nwas stronger. The Wind took the first turn. He blew with all his might to tear the traveller’s shawl \nfrom his shoulders. But the harder he blew, the tighter the traveller gripped the shawl to his body. \nThe struggle wen t on till the Wind’s turn was over. Now it was the Sun’s turn. The Sun smiled warmly. \nThe traveller felt the warmth of the smiling Sun. Soon he let the shawl fall open. The Sun’s smile grew \nwarmer and warmer... hotter and hotter. Now the traveller no longe r needed his shawl. He took it off \nand dropped it on the ground. The Sun was declared stronger than the Wind.  \nMoral:

In [6]:
pdf_chunk[1].page_content

'and dropped it on the ground. The Sun was declared stronger than the Wind.  \nMoral: Brute force can’t achieve what a gentle smile can  \n \nTHE VILLAGER AND THE SPECTACLES  \nThere was a villager. He was illiterate. He did not know how to read and write. He often saw people \nwearing spectacles for reading books or papers. He thought, “If I have spectacles, I can also read like \nthese people. I must go to town and buy a pair of sp ectacles for myself.” So one day he went to a \ntown. He entered a spectacles shop He asked the shopkeeper for a pair of spectacles for reading. The \nshopkeeper gave him various pairs of spectacles and a book. The villager tried all the spectacles one \nby one.  But he could not read anything. He told the shopkeeper that all those spectacles were \nuseless for him. The shopkeeper gave him a doubtful look. Then he looked at the book. It was upside \ndown! The shopkeeper said, “Perhaps you don’t know how to read.” The villager said, “No, I don’t. I'

In [8]:
pdf_chunk[5].metadata

{'source': 'stories.pdf', 'page': 1}

In [10]:
from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = CohereEmbeddings(
    model = "embed-english-v3.0"
)

db = FAISS.from_documents(pdf_chunk, embeddings)

In [11]:
# quering the database using similarity_search
query = "who said this: I am stronger than you "
result =db.similarity_search(query)

In [13]:
result[0].page_content

'THE WIND AND THE SUN  \nOnce the Wind and the Sun had an argument. “I am stronger than you,” said the Wind. “No, you are \nnot,” said the Sun. Just at that moment they saw a traveler walking across the road. He was wrapped \nin a shawl. The Sun and the Wind agreed that whoever could separate the traveller from his shawl \nwas stronger. The Wind took the first turn. He blew with all his might to tear the traveller’s shawl \nfrom his shoulders. But the harder he blew, the tighter the traveller gripped the shawl to his body. \nThe struggle wen t on till the Wind’s turn was over. Now it was the Sun’s turn. The Sun smiled warmly. \nThe traveller felt the warmth of the smiling Sun. Soon he let the shawl fall open. The Sun’s smile grew \nwarmer and warmer... hotter and hotter. Now the traveller no longe r needed his shawl. He took it off \nand dropped it on the ground. The Sun was declared stronger than the Wind.  \nMoral: Brute force can’t achieve what a gentle smile can'

In [14]:
from langchain_cohere import ChatCohere
model = ChatCohere(model="command-r-plus")

In [24]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """ 
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
</context>
Question: {input}

"""
)


In [25]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(model, prompt)

In [26]:
retriever = db.as_retriever()

In [27]:
from langchain.chains import create_retrieval_chain
rag_chain = create_retrieval_chain(retriever, document_chain)

In [28]:
response= rag_chain.invoke({"input": "who said this: I am stronger than you "})

In [29]:
response["answer"]

'The Wind said, "I am stronger than you."'

In [36]:
# another way of use of prompt
sytem_message = (
    """ 
You are an assistant for question answring tasks.
use the following pieces of retrieved context to answer the question.
If you don't know the answer, say that you don't know.
Answer the question in hindi language.
\n\n
{context}

"""
)

new_prompt= ChatPromptTemplate.from_messages(
    [
        ("system", sytem_message),
        ("human", "{input}")
    ]

)

In [37]:
question_answer_chain = create_stuff_documents_chain(model, new_prompt)
new_rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [38]:
new_response= new_rag_chain.invoke({"input": "what is the moral of the story: BIRBAL THE WISE"})
new_response["answer"]

'मोरल: जैसी कृति, वैसा फल'